# gmxapi sample workflow using restrained ensemble plugin

In this notebook, we will walk through a workflow in which we examine a toy system (alanine-dipeptide) with several distinct regions of conformation space, then apply a restrained ensemble biased sampling method to explore the conformational ensemble near the configuration of interest.

This system is chosen for its low computational cost and well established literature.

The biased sampling method we will use is follows a restrained ensemble technique that applies a pair restraint between selected atoms to use an (experimentally) observable pair distribution to guide MD sampling. The restraint force is a function of the difference between the target distribution and the simulated ensemble distribution. Our intent is not to promote this biasing technique for this particular system, but rather to simultaneously demonstrate a gmxapi workflow, the gmxapi MD plug-in framework, and one of the example plugin implementations included in the sample_restraint repository. The plugin was developed for simulations requiring tens of thousands of CPU hours, but these examples run in at most a few minutes on a desktop computer.

The `gmx` Python module is from the gmxapi package. The plugins built with this `sample_restraint` repository are bundled in a package named `myplugin`. While some users may find the restrained ensemble plugin useful, the repository is intended to serve as a template and starting point to develop custom pair restraint potentials. Hopefully, I have removed the least interesting name from the set of possible plugin names, and researchers are encouraged to change the name of the repository and the Python module.

A note on nomenclature: In Python lingo, `myplugin` is a Python package, a Python module, and a Python C++ extension, but these classifications are not generally equivalent. In this case, the code to calculate forces is written in C++ and built into a shared object library that can be imported into Python. Python objects created with the functions in the package can be passed through gmxapi to allow GROMACS to create local (C++ compiled binary) objects supporting high-performance MD simulation to execute a specified workflow.

In [ ]:
import sys
import os
import gmx

In [ ]:
import numpy
import subprocess
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# This only works if the gmx binary path was set in the parent process before launching the Jupyter server.
# \todo Make the docker image use the jovyan user PATH
# def find_program(program): 
#     """Return the first occurrence of program in PATH or None if not found."""
#     for path in os.environ["PATH"].split(os.pathsep):
#         fpath = os.path.join(path, program)
#         if os.path.isfile(fpath) and os.access(fpath, os.X_OK):
#             return fpath
#     return None
# gmx_path = find_program("gmx")
# if gmx_path is None:
#     gmx_path = find_program("gmx_mpi")
# if gmx_path is None:
#     raise UserWarning("gmx executable not found in path.")

In [ ]:
# Get the path to the `gmx` executable associated with the library we linked against so that we can wrap CLI tools not yet in the API.
gmx_path = os.path.join(os.environ['HOME'], 'install/gromacs/bin/gmx')
assert os.access(gmx_path, os.X_OK)

In the following cell, we set the path to the directory where some input files have been stashed.
It is a subdirectory of the `examples` directory and should contain a topology, MD parameters file, and four (previously equilibrated) atomic configurations from the same alanine-dipeptide system for independent trajectories in an ensemble simulation.

In [ ]:
# Make sure we've got access to the files we expect.
datadir = os.path.abspath('alanine-dipeptide')
workingdir = os.path.basename(datadir)
os.listdir(datadir)

gmxapi 0.0.5 requires TPR files for input, but does not have an API tool to generate them from MDP files. Wrap the command-line tool to generate run input files for the four simulations.

In [ ]:
# Turn input files into runnable binary job input.
for structure in range(4):
    structure_file = os.path.join(datadir, 'equil{}.gro'.format(structure))
    tpr_file = os.path.join(datadir, 'input{}.tpr'.format(structure))
    grompp_args = ['-c', structure_file,
                   '-o', tpr_file,
                   '-f', os.path.join(datadir, 'grompp.mdp'),
                   '-p', os.path.join(datadir, 'topol.top')]
    subprocess.call([gmx_path, "grompp"] + grompp_args)

We forumulaically generated input files above. We will load the array of four files into a specification of work. The result is a dependency graph of gmxapi operations that is nominally human-readable, but more importantly serializeable and sufficient to direct the construction of a graph of data flow and lower-level API calls to execute the intended work.

In [ ]:
tpr_files = [os.path.join(datadir, 'input{}.tpr'.format(i)) for i in range(4)]
md = gmx.workflow.from_tpr(input=tpr_files, grid=[1,1,1])

print("MD simulation element:\n\n{}".format(md.serialize()))

print("\nWork specification (pretty printed)\n")
print(str(md.workspec))

print("\nSerialized work specification\n")
print(md.workspec.serialize())

For the initial version of this walk-through, we have not chosen or implemented a way to execute the 4-rank simulation ensemble to perform this work. We can run a single ensemble member (below) or we can resort to a Python script in this same directory. From `sample_restraint/examples`, run `mpiexec -n 4 python -m mpi4py example.py` to run the 4-member ensemble and generate the data for the first Ramachandran plot.

In [ ]:
# We don't currently have a way of running an array of jobs from a jupyter notebook.
#
#with gmx.context.ParallelArrayContext(md) as session:
#    session.run()

In [ ]:
tpr_files = [os.path.join(datadir, 'input{}.tpr'.format(i)) for i in range(4)]
md = gmx.workflow.from_tpr(input=[tpr_files[0]], grid=[1,1,1], threads=2, pme_ranks=1, tmpi=2)

my_context = gmx.context.ParallelArrayContext(md)

with my_context as session:
    session.run()


In [ ]:
# Wrap the gmx tool to extract phi and psi values for a Ramachandran diagram. E.g.
# ~/gromacs-mpi/bin/gmx_mpi rama -s topol.tpr -f traj_comp.xtc
def rama(run_input=None, trajectory=None, output="rama.xvg", executable=gmx_path):
    """Use the GROMACS tool to extract psi and phi angles for the provided structure and trajectory."""
    if run_input is not None and trajectory is not None and output is not None and executable is not None:
        for file_arg in [run_input, trajectory]:
            if not os.path.exists(file_arg):
                raise FileNotFoundError("Invalid file: {}".format(file_arg))
        if not os.access(gmx_path, os.X_OK):
            raise FileExistsError("Invalid executable: {}".format(gmx_path))
    else:
        raise RuntimeError("Bad arguments.")
    subprocess.call([gmx_path, "rama", "-s", run_input, "-f", trajectory, "-o", output])

In [ ]:
run_input = os.path.join(datadir, 'input0.tpr')
trajectory = os.path.join(my_context.workdir, 'traj_comp.xtc')
trajectory = os.path.join(my_context.workdir, 'traj.trr')

rama_file = os.path.join(my_context.workdir, 'rama.xvg')
rama(run_input=run_input, trajectory=trajectory, output=rama_file)

In [ ]:
phi, psi = numpy.genfromtxt(rama_file, skip_header=13, comments='@', usecols=(0,1)).T

In [ ]:
fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
ax.scatter(phi, psi)
ax.set_xlim(-180, 180)
ax.set_ylim(-180, 180)
ax.set_xlabel('phi')
ax.set_ylabel('psi')
ax.set_title('Alanine dipeptide Ramachandran plot')

In [ ]:
plt.plot(phi, '.', label='phi')
plt.plot(psi, '.', label='psi')
plt.legend()
plt.title("Evolution of phi and psi")

In [ ]:
import MDAnalysis
import MDAnalysis.analysis.distances

In [ ]:
u = MDAnalysis.Universe(os.path.join(datadir, 'equil0.gro'), trajectory)

In [ ]:
print(u.residues)

In [ ]:
alanine = u.select_atoms("resname ACE ALA NME")

In [ ]:
alanine

In [ ]:
trajs = numpy.array([MDAnalysis.analysis.distances.self_distance_array(alanine.positions) for _ in u.trajectory])

In [ ]:
trajs.shape

In [ ]:
for traj in trajs:
    plt.plot(traj)
plt.ylim(0,10)

In [ ]:
potential = gmx.workflow.WorkElement(namespace="myplugin",
                                     operation="ensemble_restraint",
                                     params=[1, 4, 2.0, 10000.0])
potential.name = "restrained_ensemble"
md.add_dependency(potential)

In [ ]:
print(md.serialize())

In [ ]:
print(md.workspec)

In [ ]:
md = gmx.workflow.from_tpr(tpr_filename)
md.add_dependency(potential)


In [ ]:
potential.workspec = None
md.add_dependency(potential)

In [ ]:
context = gmx.context.ParallelArrayContext(md)
with context as session:
    if context.rank == 0:
        print(context.work)
    status = session.run()
print(status)